In [ ]:
#export
from data_lab.imports import *
import math # For testing
import getpass

ModuleNotFoundError: No module named 'lab'

In [ ]:
#default_exp develop.utils

# Develop Utilities

Establish some interfaces for developing work.

* Pass/Fail and format validation steps.
* ValidateCode: Ensures the code should dispatch to an environment
* ValidationRecord: ensure the record is ready to be stored


In [ ]:
#export

def listify(o):
    """Create lists from objects"""
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, dict): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

PASSING_STATUS = 'passed'
FAILING_STATUS = 'failed'

def _create_step(name, message=None, status=PASSING_STATUS, **kw):
    """Create a step. A step is just a dictionary with some conventions.
    Default to passed and no message."""
    d = {**kw, **dict(name=name, status=status)}
    if not message is None: d['message'] = message
    return d

_pass_step = _create_step # Default is pass

def _fail_step(name, exception=None, **kw):
    """Create a failing step using an exception.
    It's common to pass a traceback to this function, but it's not
    necessary, or even used in the current state of anything."""
    return _create_step(name, status=FAILING_STATUS, message=str(exception), **kw)


In [ ]:
assert listify(None) == []
assert listify(['a']) == ['a']
assert listify('a') == ['a']
assert listify({'a': 1}) == [{'a': 1}]

passing = _pass_step('foo')
assert passing['name'] == 'foo'
assert passing['status'] == PASSING_STATUS
assert _create_step('foo') == passing

exception = None
trace = None
try:
    raise AssertionError("Exception message")
except Exception as e:
    _, _, trace = sys.exc_info()
    exception = e
failing = _fail_step('foo', exception=exception, traceback=trace)
assert failing['name'] == 'foo'
assert failing['status'] == FAILING_STATUS
assert failing['traceback'] == trace
assert failing['message'] == 'Exception message'

In [ ]:
#export

class FormatValidation:
    """convert a list of steps into a more-reasonable string explanation
    of what took place."""
    
    @classmethod
    def call(cls, steps, **kw):
        return cls(steps, **kw)()
    
    def __init__(self, steps, **kw):
        self.steps = steps
        self.kw = kw
        
    @property
    def all_passing(self):
        if hasattr(self, '_all_passing'): return self._all_passing
        self._all_passing = all([v == PASSING_STATUS for v in self._get_key('status')])
        return self._all_passing
        
    @property
    def valid(self):
        if hasattr(self, '_valid'): return self._valid
        self._valid = self.kw.get('valid', self.all_passing)
        return self._valid
    
    @property
    def passed(self):
        if hasattr(self, '_passed'): return self._passed
        self._passed = self._filter_steps(key='status', value=PASSING_STATUS)
        return self._passed
    
    @property
    def passing_message(self):
        if hasattr(self, '_passing_message'): return self._passing_message
        messages = []
        messages.append("Validation was successful.")
        passed_names = self._get_key('name', steps=self.passed)
        if len(passed_names) == 0:
            messages.append("No successful steps were reported.")
        else:
            step_names = ', '.join(passed_names)
            messages.append(f"Steps: {step_names}")
        self._passing_message = "\n".join(messages)
        return self._passing_message
    
    @property
    def failing_message(self):
        if hasattr(self, '_failing_message'): return self._failing_message
        messages = []
        messages.append("Validation failed.")

        for step in self.steps:
            messages.append(self._explain_step(step))
        
        failed_steps = self._filter_steps(value=FAILING_STATUS)
        if len(failed_steps) == 0: messages.append("No failing steps were reported.")
        self._failing_message = "\n".join(messages)
        return self._failing_message
    
    @property
    def message(self):
        if hasattr(self, '_message'): return self._message
        if self.valid:
            self._message = self.passing_message
        else:
            self._message = self.failing_message
        return self._message
        
    def _filter_steps(self, key='status', value=None, **kw):
        """Utility to filter a list of dictionaries by a key."""
        return [d for d in self.steps if d.get(key) == value]
    
    def _get_key(self, key, steps=None, **kw):
        """Utility to get a value from a list of dictionaries by a key."""
        if steps is None: steps = self.steps
        return [e.get(key) for e in steps]
    
    def _explain_success(self, step, default_name='unknown'):
        """Format a single successful step."""
        name = step.get('name', default_name)
        return f"Success with: {name}"
    
    def _explain_failure(self,
                         step,
                         default_name='unknown',
                         default_reason='reason unknown'):
        """Format a single failing step."""
        name = step.get('name', default_name)
        reason = step.get('message', default_reason)
        return f"Failing step: {name}. Reason: {reason}."
    
    def _explain_malformed(self, step, default_name='unknown'):
        name = step.get('name', default_name)
        reason = step.get('message')
        message = f"Possible problem with {name}. The status was not set, but assumed to be a problem."
        if not reason is None: message += f' Message: {reason}'
        return message
    
    def _explain_step(self, step):
        status = step.get('status')
        if status == PASSING_STATUS: return self._explain_success(step)
        if status == FAILING_STATUS: return self._explain_failure(step)
        return self._explain_malformed(step)
    
    def __call__(self):
        return self.message
    
def format_validation(steps, **kw): return FormatValidation.call(steps, **kw)

In [ ]:
steps = [
    dict(name='a', status='passed'),
    dict(name='b', status='passed'),
    dict(name='c', status='failed', message='Explain problems with c'),
    dict(name='d', status='passed'),
    dict(name='e'),
    dict(name='f', status='failed'),
    dict(status='failed', message="Something went wrong in a malformed step"),
]

subject = FormatValidation(steps)
assert not subject.valid
assert subject.message == subject.failing_message
assert subject() == subject.message
assert "Validation failed" in subject()
assert "Success with: a" in subject()
assert "Failing step" in subject()
assert "Possible problem" in subject()

assert "Validation was successful" in FormatValidation.call(steps, valid=True)
assert "No successful steps were reported" in FormatValidation.call([])
assert FormatValidation([]).valid
assert "No successful steps were reported" in FormatValidation.call([{'status': FAILING_STATUS}], valid=True)
assert "Validation was successful" in format_validation([])

In [ ]:
#export

class ValidateCode:
    """Validate code works. This is currently defaulted for a Treatment,
    but as I develop other tools, I'll come up with a more generic way to
    to do this."""
    
    DEFAULT_STEPS = [
        ('setup model', 'fn'),
        ('configure model', 'model'),
        ('duck type', 'valid_duck_type')
    ]
    
    @classmethod
    def call(cls, **kw):
        return cls(**kw)()
    
    def __init__(self, **kw):
        self.kw = kw
        
    @property
    def steps(self):
        if hasattr(self, '_steps'): return self._steps
        self._steps = self.kw.get('steps', self.DEFAULT_STEPS)
        return self._steps
        
    @property
    def name(self):
        """Get the name."""
        if hasattr(self, '_name'): return self._name
        self._name = self.kw.get('name')
        return self._name
    
    @property
    def package(self):
        """Reveal the string of a package, if provided."""
        if hasattr(self, '_package'): return self._package
        self._package = self.kw.get('package')
        return self.package
    
    @property
    def loaded_package(self):
        """Reveal the imported package, if provided."""
        if hasattr(self, '_loaded_package'): return self._loaded_package
        if self.package is None: return None
        try:
            self._loaded_package = __import__(self.package)
            return self._loaded_package
        except:
            self._loaded_package = None
        
    @property
    def fn(self):
        if hasattr(self, '_fn'): return self._fn
        g = globals()
        if 'fn' in self.kw and self.kw['fn'] in g:
            self._fn = g[self.kw['fn']]
        if self.name:
            if self.name in g:
                self._fn = g[self.name]
            if self.loaded_package and hasattr(self.loaded_package, self.name):
                self._fn = getattr(self.loaded_package, self.name)
        if not hasattr(self, '_fn'): self._fn = None
        return self._fn
    
    @property
    def unused_keywords(self):
        """Sloppy way to use keywords for both validation and model building."""
        if hasattr(self, '_unused_keywords'): return self._unused_keywords
        known = ['steps', 'name', 'package'] # keywords used in validation, not model building
        self._unused_keywords = {k:v for k, v in self.kw.items() if not k in known}
        return self._unused_keywords
    
    @property
    def model(self):
        """Construct a model from the unused keywords."""
        # TODO: Revisit this, it might not be what I want.
        if hasattr(self, '_model'): return self._model
        self._model = self.fn()(**self.unused_keywords)
        return self._model
    
    @property
    def valid_duck_type(self):
        """TODO: write validations that the model is usable."""
        return True
    
    def validate(self):
        """Run the steps to validate."""
        results = []
        for (name, attribute) in self.steps:
            try:
                passing = bool(getattr(self, attribute))
                result = _pass_step(name) if passing else _fail_step(name)
                results.append(result)
            except Exception as e:
                _, _, exc_traceback = sys.exc_info()
                results.append(_fail_step(name, exception=e, traceback=exc_traceback))
        passing = all([d.get('status') == PASSING_STATUS for d in results])
        return passing, results
    
    def __call__(self):
        return self.validate()
    
def validate_code(**kw): return ValidateCode.call(**kw)

In [ ]:
class _SpecialTreatment:
    """Treatment used for testing."""
    
    def model(self, **kw):
        return 'model...that needs to duck type'
    
    def __call__(self, **kw):
        return self.model(**kw)

base = dict(
    name='_SpecialTreatment'
)

subject = ValidateCode(**base)
assert subject.name == base['name']
assert subject.fn == _SpecialTreatment
assert subject.steps == ValidateCode.DEFAULT_STEPS

subject = ValidateCode(name='acos', package='math')
assert subject.name == 'acos'
assert subject.package == 'math'
assert subject.loaded_package == math
assert subject.fn == math.acos

model_keywords = {'tools': 'for the model'}
subject = ValidateCode(name='_SpecialTreatment', **model_keywords)
assert subject.unused_keywords == model_keywords
expected = _SpecialTreatment()(**model_keywords)
assert subject.model == expected
passed, steps = subject()
assert passed

passed, steps = validate_code(name='acos', package='math')
assert not passed
explanation = format_validation(steps)
assert "Failing step: configure model" in explanation

At this point we can validate some code. As mentioneed above, it's still specific to a treatment, with the exception of duck typing the treatment--will it work with other tools? That's coming.

I don't like the way I'm building the model, sharing keywords, sharing details at high levels. Given some use of treatments, I've got to redesign some interfaces.

Also, it's probably OK that I create classes for the heavy lifting, but I've been hesitant to bring load_once and load_once_property into this library. Keeping it simple is pedantic.

Some things will move out to a utilities module:

* FormatValidation
* _pass_step and _fail_step
* Some of the dictionary utilities in FormatValidation

In [ ]:
#export

class ValidateRecord:
    """Given the state of a record, the data we record around the code,
    is it valid? ValidateCode deals with whether the validation should work
    with our environment. This code deals with whether we're setting the right
    attributes in the database."""
    
    BASIC_PROPERTIES = ['name', 'version', 'contributors', 'state', 'created_at', 'updated_at']
    REQUIRED_MAP = dict(
        started = BASIC_PROPERTIES,
        revised = BASIC_PROPERTIES,
        validated = BASIC_PROPERTIES + ['class'],
        production = BASIC_PROPERTIES + ['class'],
        retired = BASIC_PROPERTIES,
    )

    ADDITIONAL_PROPERTIES = ['package', 'description', 'target_release']
    
    ALLOWED_MAP = dict(
        started = REQUIRED_MAP['started'] + ADDITIONAL_PROPERTIES,
        revised = REQUIRED_MAP['revised'] + ADDITIONAL_PROPERTIES,
        validated = REQUIRED_MAP['validated'] + ADDITIONAL_PROPERTIES,
        production = REQUIRED_MAP['production'] + ADDITIONAL_PROPERTIES,
        retired = REQUIRED_MAP['retired'] + ADDITIONAL_PROPERTIES + ['replacement'],
    )
    
    DEFAULT_VERSION = dict(
        major = 0,
        minor = 0,
        patch = 1,
    )
    
    @classmethod
    def call(cls, **kw):
        return cls(**kw)()
    
    def __init__(self, **kw):
        self.kw = kw
    
    @property
    def state(self):
        return self.kw.get('state', list(self.REQUIRED_MAP.keys())[0])
    
    @property
    def config(self):
        if hasattr(self, '_config'): return self._config
        self._config = self.kw.get('config', {})
        return self._config
    
    @property
    def user(self):
        if hasattr(self, '_user'): return self._user
        self._user = self.config.get('user', getpass.getuser())
        return self._user
    
    @property
    def contributors(self):
        if hasattr(self, '_contributors'): return self._contributors
        contributors = listify(self.kw.get('contributors', []))
        if not self.user in contributors: contributors += [self.user]
        self._contributors = contributors
        return self._contributors

    @property
    def version(self):
        if hasattr(self, '_version'): return self._version
        version = self.kw.get('version', self.DEFAULT_VERSION)
        if isinstance(version, str):
            d = dict(enumerate(version.split('.')))
            version = dict(
                major = d.get(0, self.DEFAULT_VERSION['major']),
                minor = d.get(1, self.DEFAULT_VERSION['minor']),
                patch = d.get(2, self.DEFAULT_VERSION['patch']),
            )
        self._version = version
        return self._version
    
    @property
    def now(self):
        if hasattr(self, '_now'): return self._now
        self._now = self.kw.get('now', datetime.utcnow())
        return self._now
    
    @property
    def created_at(self):
        return self.kw.get('created_at', self.now)
    
    @property
    def updated_at(self):
        return self.kw.get('updated_at', self.now)

    def _has_field(self, key):
        return hasattr(self, key) or key in self.kw
    
    def _get_field(self, key):
        if hasattr(self, key): return getattr(self, key)
        return self.kw.get(key)
    
    def _get_fields(self, keys):
        return {key:self._get_field(key) for key in keys if self._has_field(key)}
    
    @property
    def attributes(self):
        """The available attributes for the state."""
        if hasattr(self, '_attributes'): return self._attributes
        assert self.state in self.ALLOWED_MAP, f"Invalid state: {self.state}. Must by one of {list(self.ALLOWED_MAP.keys())}"
        self._attributes = self._get_fields(self.ALLOWED_MAP[self.state])
        return self._attributes
    
    @property
    def state_based_conditions(self):
        validate_states = ['validated', 'production']
        if not self.state in validate_states: return True
        validator = Validate(**self.attributes)
        valid, _ = validator()
        return valid
    
    @property
    def valid(self):
        """Do we have the required attributes for the state? """
        if hasattr(self, '_valid'): return self._valid
        assert self.state in self.REQUIRED_MAP, f"Invalid state: {self.state}. Must by one of {list(self.REQUIRED_MAP.keys())}"
        found = [key in self.attributes for key in self.REQUIRED_MAP[self.state]]
        self._valid = all(found) and self.state_based_conditions
        return self._valid
    
    def __call__(self):
        return self.valid, self.attributes
    
def validate_records(**kw): return ValidateRecord.call(**kw)

In [ ]:
subject = ValidateRecord()
assert subject.state == list(ValidateRecord.ALLOWED_MAP.keys())[0]
assert not subject.valid

now = datetime.utcnow()
basic_values = dict(
    name = 'foo',
    version = '1.2.3',
    contributors = 'someone_else',
    now = now,
)
subject = ValidateRecord(**basic_values)
assert subject._get_field('name') == 'foo'
assert subject.version['major'] == '1'
assert subject.version['minor'] == '2'
assert subject.version['patch'] == '3'
assert subject.contributors == ['someone_else', subject.user]
assert subject.created_at == now
assert subject.updated_at == now
assert subject.valid
valid, attributes = subject()
assert valid
assert attributes == subject.attributes

with check_raises(message='Invalid state raises...is this what I want?'):
    subject = ValidateRecord(state='not a state')
    subject.valid
    
subject = ValidateRecord(state='validated', **basic_values)
assert subject.state == 'validated'
assert not subject.valid

In [ ]:
#export

def store(object_store=noop_object_store, **kw):
    """Store the record, if valid, in the provided object store. Defaults to a noop data store."""
    valid, record = ValidateRecord.call(**kw)
    if not valid:
        return False
    return object_store.put(item=record, **record)

def start(**kw):
    """Start a new record, ensuring the state is correct."""
    kw = {**kw, **{'state': 'started'}}
    return store(**kw)

def increment_version(version, level='patch', **kw):
    levels = ['major', 'minor', 'patch']
    if not level in levels: level = 'patch'
    if version is None: version = "0.0.0"
    if isinstance(version, str):
        d = dict(enumerate(version.split('.')))
        version = dict(
            major = d.get(0, 0),
            minor = d.get(1, 0),
            patch = d.get(2, 0),
        )
    assert isinstance(version, dict), "Version must be a dictionary or convertible to a dictionary."
    version = {k:int(version.get(k, 0)) for k in levels}
    version[level] += 1
    if level == 'major':
        version['minor'] = 0
        version['patch'] = 0
    if level == 'minor':
        version['patch'] = 0
    return version

def string_version(version, **kw):
    "Convert a dictionary version to a string format"
    if isinstance(version, str): return version
    if isinstance(version, dict):
        return f"{version.get('major', 0)}.{version.get('minor', 0)}.{version.get('patch', 0)}"
        
def revise(name, item=None, object_store=noop_object_store, **kw):
    """Revise a record, using the name to find the old record, if it exists."""
    if item is None: item = {'name': name} # For testing
    now = kw.pop('now', datetime.utcnow())
    record = object_store.get(name=name, item=item)
    old_version = record.get('version', None)
    new_version = increment_version(old_version)
    record['version'] = new_version
    record['state'] = 'revised'
    record['updated_at'] = now
    kw = {**record, **kw, **{'object_store': object_store}}
    return store(**kw, item=record)

In [ ]:
assert string_version(increment_version('0.1.1')) == '0.1.2'
assert string_version(increment_version('0.1.1', level='minor')) == '0.2.0'
assert string_version(increment_version('0.1.1', level='major')) == '1.0.0'

assert isinstance(store(name='foo'), dict)
assert not store() # Not valid, no name

version = start(name='foo', version='0.1.0')['version']
assert string_version(version) == '0.1.0'

previous = datetime.utcnow() - dt.timedelta(days=1)
now = datetime.utcnow()
existing = start(name='foo', contributors='someone_else', now=previous)
assert string_version(existing['version']) == '0.0.1'
revised = revise(name='foo', item=existing, now=now)
assert revised['state'] == 'revised'
assert revised['updated_at'] == now
assert 'someone_else' in revised['contributors']
assert getpass.getuser() in revised['contributors']
assert string_version(revised['version']) == '0.0.2'